<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/Project/langchain_meta_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LangSmith : https://smith.langchain.com/

In [3]:
!pip install langchain-openai
!pip install langchain
!pip install -U langchain-community
# !pip install langchainhub
!pip install -qU huggingface_hub
!pip install langchain_huggingface

In [21]:
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [7]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "donghuna_proj"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANG_SMITH')


In [9]:
# system_prompt = """당신은 표(재무제표) 를 해석하는 금융 AI 어시스턴트 입니다.
# 당신의 임무는 주어진 테이블 형식의 재무제표를 바탕으로 흥미로운 사실을 정리하여 친절하게 답변하는 것입니다."""

# # 질의내용
# question = "대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!"

# prompt = ChatPromptTemplate.from_messages([
#     ("system", system_prompt),
#     ("user", "{input}")
# ])

# # 객체 생성
# llm = ChatOpenAI(
#     model_name="gpt-4o-mini",  # 모델명
#     api_key=userdata.get('OPEN-AI_KEY'),
#     organization=userdata.get('MY_OPENAI_ORG_ID'),
#     streaming=False,
#     temperature=0.2  # 창의성 (0.0 ~ 2.0)
# ).bind(logprobs=True)

# output_parser = StrOutputParser()

# chain = prompt | llm | output_parser
# chain.invoke(question)

# 질의
# answer = llm.stream(question)
# for token in answer:
#     print(token.content, end="", flush=True)
# response = llm.invoke(question)
# print(response.content)
# print(response.response_metadata)



스트리밍 출력

In [11]:
# print(f"ChatOpenAI: {chain.is_lc_serializable()}")

In [13]:
from huggingface_hub import login

login(token=userdata.get('HUG_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [15]:
from langchain.prompts import PromptTemplate

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate.from_template(template)

Huggingface Endpoint

direct 접근방법

In [17]:
# from langchain_huggingface import HuggingFaceEndpoint

# # 사용할 모델의 저장소 ID를 설정합니다.
# # repo_id = "microsoft/Phi-3-mini-4k-instruct"
# repo_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# # repo_id = "donghuna/Meta-Llama-3.1-8B-Instruct"

# llm = HuggingFaceEndpoint(
#     repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
#     max_new_tokens=256,  # 생성할 최대 토큰 길이를 설정합니다.
#     temperature=0.1,
#     huggingfacehub_api_token=userdata.get('HUG_TOKEN'),  # 허깅페이스 토큰
# )

# output_parser = StrOutputParser()

# # LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
# chain = prompt | llm | StrOutputParser()
# # 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.
# response = chain.invoke({"question": "안녕하세요? transformer 모델에 대해서 설명해주세요"})
# print(response)

In [18]:
!pip install --upgrade transformers

In [23]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_id = "donghuna/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    # model_kwargs={"torch_dtype": torch.float32},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "안녕하세요? transformer 모델에 대해서 설명해주세요"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "Yer lookin' fer a tale about transformer 모델, eh? Alright then, settle yerself down with a pint o' grog and listen close.\n\nTransformer 모델이란, 자연어 처리에 큰 변화를 가져온 모델 중 하나야. 이 모델은 Attention Mechanism 이라는 기법을 사용해, 입력 문장의 모든 단어에 대해同時 attention 을 계산할 수 있어, 문맥에 대한 이해를 높일 수 있어.\n\nTransformer 모델은 Encoder-Decoder 구조를 사용해, 입력 문장과 출력 문장을 처리해. Encoder는 입력 문장을 받아, 고차원 벡터로 변환하고, Decoder는 출력 문장을 생성해.\n\nAttention Mechanism 이란, 입력 문장의 각 단어에 대해, 다른 단어와의 관계를 계산해, 중요도가 높은 단어에 더 많은 attention 을 주는 방식이야. 이를 통해, 문맥에 대한 이해를 높일 수 있어.\n\nTransformer 모델은 기존의 RNN 모델보다, 더 빠르고, 더 효율적으로 작동해. 또한, 문맥에 대한 이해를 높일 수 있어, 다양한 자연어 처리 작업에 적합해.\n\nSo hoist the sails and set course fer a world"}


In [ ]:
messages = [
    # {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "너 뭐야 왜케 말잘해"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
messages = [
    # {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "프롬프트 엔지니어링에 대해서 설명해줘. 그리고 어떤 프롬프트가 좋은 프롬프트인지 예시를 알려줘"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_id = "donghuna/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)



In [ ]:
device = 0 if torch.cuda.is_available() else -1

# 텍스트 생성 파이프라인 초기화
generator = pipeline('text-generation',
                     model=model,
                     tokenizer=tokenizer,
                     pad_token_id=tokenizer.eos_token_id,
                     device=device)

# 텍스트 생성 예제
text = "Once upon a time,"
generated_texts = generator(text)
print(generated_texts)

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Hello?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

모델 로드 및 푸시하기